In [1]:
# from models.detection.ofa_resnet50_fasterrcnn import get_ofa_resnet50_fasterrcnn_model, load_pretrained_fasterrcnn, set_training_params
# model = get_ofa_resnet50_fasterrcnn_model()
# print(model)

from models.detection.ofa_mbv3_w12_fcos import get_ofa_mbv3_w12_fcos_model, load_pretrained_fcos, set_training_params
model = get_ofa_mbv3_w12_fcos_model()
print(model)

Using cache found in C:\Users\wydai/.cache\torch\hub\mit-han-lab_once-for-all_master


FCOS(
  (backbone): Mbv3W12FcosFpn(
    (body): OFAMobileNetV3(
      (first_conv): ConvLayer(
        (conv): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): Hswish()
      )
      (blocks): ModuleList(
        (0): ResidualBlock(
          (conv): MBConvLayer(
            (depth_conv): Sequential(
              (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=24, bias=False)
              (bn): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (act): ReLU(inplace=True)
            )
            (point_linear): Sequential(
              (conv): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
          )
          (shortcut): IdentityLayer()
        

In [2]:
# 只保存部分权重
weight_to_save = ['backbone.dynamic_convs', 'backbone.fpn', 'roi_heads', 'head']

In [3]:
import torch

# 保存指定的层
def save_selected_layers(model, layers_to_save, save_path):
    state_dict = {}
    for name, param in model.named_parameters():
        # 检查该参数是否属于我们要保存的层
        if any(layer in name for layer in layers_to_save):
            state_dict[name] = param
    torch.save(state_dict, save_path)

# 加载指定的层
def load_selected_layers(model, checkpoint_path):
    state_dict = torch.load(checkpoint_path)
    model_state_dict = model.state_dict()
    
    # 只更新checkpoint中存在的层
    for name, param in state_dict.items():
        if name in model_state_dict:
            model_state_dict[name] = param
            print(f'Loaded {name}')
    
    model.load_state_dict(model_state_dict)


In [4]:
save_path = 'selected_layers.pth'
save_selected_layers(model, weight_to_save, save_path)
load_selected_layers(model, save_path)

Loaded backbone.fpn.inner_blocks.0.0.weight
Loaded backbone.fpn.inner_blocks.0.0.bias
Loaded backbone.fpn.inner_blocks.1.0.weight
Loaded backbone.fpn.inner_blocks.1.0.bias
Loaded backbone.fpn.inner_blocks.2.0.weight
Loaded backbone.fpn.inner_blocks.2.0.bias
Loaded backbone.fpn.inner_blocks.3.0.weight
Loaded backbone.fpn.inner_blocks.3.0.bias
Loaded backbone.fpn.layer_blocks.0.0.weight
Loaded backbone.fpn.layer_blocks.0.0.bias
Loaded backbone.fpn.layer_blocks.1.0.weight
Loaded backbone.fpn.layer_blocks.1.0.bias
Loaded backbone.fpn.layer_blocks.2.0.weight
Loaded backbone.fpn.layer_blocks.2.0.bias
Loaded backbone.fpn.layer_blocks.3.0.weight
Loaded backbone.fpn.layer_blocks.3.0.bias
Loaded backbone.dynamic_convs.0.conv.weight
Loaded backbone.dynamic_convs.1.conv.weight
Loaded backbone.dynamic_convs.2.conv.weight
Loaded backbone.dynamic_convs.3.conv.weight


In [41]:
from thop import profile
import torch
import torch.nn as nn

subnet_config = model.backbone.body.sample_active_subnet()
model.backbone.body.set_active_subnet(**subnet_config)
# model.backbone.body.set_max_net()

# from models.backbone.ofa_supernet import get_ofa_supernet_mbv3_w12
# model = get_ofa_supernet_mbv3_w12()

# subnet_config = model.sample_active_subnet()
# model.set_active_subnet(**subnet_config)

print(subnet_config)
input = torch.randn(1, 3, 128, 128)
flops, params = profile(model, inputs=(input, ))
print(f"Total FLOPs: {flops}")

{'ks': [7, 7, 5, 3, 7, 3, 5, 7, 3, 7, 5, 7, 3, 7, 3, 7, 5, 3, 7, 3], 'e': [6, 6, 6, 3, 4, 4, 6, 3, 3, 6, 4, 3, 3, 4, 3, 4, 4, 6, 6, 3], 'd': [4, 3, 3, 2, 4]}
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
Total FLOPs: 74616492800.0
